In [ ]:
# Standard Library imports first
import time
from functools import lru_cache as cache
from os.path import isdir, isfile
from os import makedirs
from random import randint
from time import gmtime, strftime

# 3rd Party imports, light-show next
from logzero import logger, setup_logger
from pyfiglet import figlet_format
from colorama import Fore

# Specialized tricksters last
import requests
import numpy as np
import pandas as pd
import pipulate as gs

from datetime import date, datetime, timedelta

# Start logging right away
if not isdir('./logs/'):
    makedirs('./logs/')
logger = setup_logger(logfile='./logs/app.log', maxBytes=1000000, backupCount=30)
logger.info("The first order of business is accountability. Make it sparkle!")

def h1(print_me, color=Fore.GREEN, font='standard'):
    ''' Never underestimate the power of a good nickname. '''
    ascii_art = figlet_format(print_me, font=font)
    print('%s%s%s' % (color, ascii_art, Fore.WHITE))

def h2(print_me, color=Fore.GREEN, font='cybermedium'):
    ''' The human brain generally only deals well with shallow nesting. '''
    ascii_art = figlet_format(print_me, font=font)
    print('%s%s%s' % (color, ascii_art, Fore.WHITE))

h1('Pipulating')

sheet = gs.key('1U-7JPK6Zk4sVApjZpjWSAMSl99-N0gNKxMXRiagigr8')
tab_name = 'Sheet1'
tab = sheet.worksheet(tab_name)

row1 = tab.row_values(1)
if row1:
    while not row1[-1]:
        row1.pop()
else:
    logger.error("Row1 can not be empty. Exiting.")
    raise SystemExit()

fm = dict()
fn = list()
for i, item in enumerate(row1):
    fm[item] = eval("'%s'" % gs.cc(i+1))
    fn.append(item)

col_count = len(row1)
col1 = tab.col_values(1)

if col1:
    while not col1[-1]:
        col1.pop()
else:
    logger.error('You must have at least one row of data')
    raise SystaemExit()

try:
    first_row_with_data = col1.index('formatting') + 2
except:
    logger.error('Column A must have a cell containing "formatting" immediately above first data row.')

logger.info('Auto-trimming extra rows & columns.')
tab.resize(rows=len(col1), cols=col_count+1)
tab.resize(rows=len(col1)+2, cols=col_count+1)

logger.info('Updating date range labels.')
dates = gs.date_ranges(yoy=False)
human_dates = gs.date_ranges(human=True, yoy=False)
if 'date range' in col1:
    range_row = col1.index('date range') + 1
    for metric in ['users', 'revenue']:
        for i, num in enumerate(['90', '60', '30']):
            try:
                field_dex = '%s%s' % (metric, num)
                range_col = fn.index(field_dex) + 1
                tab.update_cell(range_row, range_col, human_dates[i])
            except:
                pass

logger.info('Auto-selecting range from Google Sheet.')            
rows = first_row_with_data, len(col1)
cols = 'A', gs.cc(len(row1))
cl, df = gs.pipulate(tab, rows, cols)

@cache(maxsize=128)
def randomcache():
    return randint(1,100)

logger.info('Processing each row with timestamps.')            
for item in fm:
    if item != 'index':
        eval_me = 'fm["%s"]' % item
        column_letter = eval(eval_me)
        df[column_letter] = randomcache()
        timestamp_row = col1.index('timestamp') + 1
        timestamp_col = gs.aa(column_letter)
        gmt = strftime("%a, %d %b %Y %H:%M GMT", gmtime())
        tab.update_cell(timestamp_row, timestamp_col, gmt)

logger.info('Pushing data back to GSheet.')
gs.populate(tab, cl, df)